In [2]:
%pip install -q \
  antropy \
  openpyxl \
  pandas \
  scipy

Note: you may need to restart the kernel to use updated packages.


In [3]:
# set up the environment
import pandas as pd
pd.set_option('display.max_columns', 128)

In [77]:
TRANSFORMED_DATA_OUTPUT_NAME = '../datasets/swell/data/final/test-custom-1'

In [5]:
# prepare the raw data
from pathlib import Path
import pandas as pd
from zipfile import ZipFile

# extract the compressed raw data files
raw_data_dir = '../datasets/swell/data/raw'
if Path(raw_data_dir).exists():
  print(f'Raw data files were already extracted')
else:
  ZipFile(f'{raw_data_dir}.zip', 'r').extractall('..')
  print(f'Raw data files have been extracted')
# load the data
print(f'Loading data files')

# combine the label data
LABEL_DATA = (
  pd.concat(
    copy=False,
    objs=pd.read_excel(
      io=f'{raw_data_dir}/labels/hrv stress labels.xlsx',
      index_col=0,
      sheet_name=None,
    ),
  )[['subject', 'ElapsedTime', 'label']]
  .set_index(['subject', 'ElapsedTime'])
  .sort_index()
)
LABEL_DATA['label'] = LABEL_DATA['label'].astype('category')
print(f'Label data has been loaded:')
display(LABEL_DATA)

# combine the RRI data
RRI_DATA = []
for rri_data_path in Path(raw_data_dir).glob('rri/*.csv'):
  data = pd.read_csv(rri_data_path)
  data['subject'] = rri_data_path.stem
  RRI_DATA.append(data.rename(columns={'Time': 'ElapsedTime (sec)'}))
RRI_DATA = pd.concat(objs=RRI_DATA, copy=False)
RRI_DATA['ElapsedTime'] = (RRI_DATA['ElapsedTime (sec)'] / 60).astype(int)
RRI_DATA.set_index(['subject', 'ElapsedTime'], inplace=True)
RRI_DATA.sort_index(inplace=True)

print(f'RR interval data has been loaded:')
display(RRI_DATA)

Raw data files were already extracted
Loading data files
Label data has been loaded:


label
subject ElapsedTime              
p1      0                    rest
        1                    rest
        2                    rest
        3                    rest
        4                    rest
...                           ...
p9      158          interruption
        159          interruption
        160          interruption
        161          interruption
        162          interruption

[3140 rows x 1 columns]

RR interval data has been loaded:


ElapsedTime (sec)        rri
subject ElapsedTime                              
p1      0                     1.265625  870.11719
        0                     1.515625  885.36996
        0                     1.765625  890.18974
        0                     2.015625  886.73851
        0                     2.265625  877.17820
...                                ...        ...
p9      127                7644.475100  884.87308
        127                7644.725100  903.45423
        127                7644.975100  920.95615
        127                7645.225100  936.70456
        127                7645.475100  950.02517

[704454 rows x 2 columns]

In [6]:
# merge label and RRI data
MERGED_DATA = (pd.merge(
    left=LABEL_DATA,
    right=RRI_DATA,
    left_index=True,
    right_index=True,
    how='inner',
    copy=False,
  ).droplevel(level='ElapsedTime', axis=0)
  .sort_index(axis=1))
MERGED_DATA = MERGED_DATA[MERGED_DATA['label'] != 'rest']
MERGED_DATA['label'] = MERGED_DATA['label'].cat.remove_unused_categories()
MERGED_DATA

,ElapsedTime (sec),label,rri
subject,,,
p1,600.01563,no stress,732.20732
p1,600.26563,no stress,709.33594
p1,600.51563,no stress,690.18264
p1,600.76563,no stress,683.69979
p1,601.01563,no stress,687.40428
...,...,...,...
p9,7644.47510,interruption,884.87308
p9,7644.72510,interruption,903.45423
p9,7644.97510,interruption,920.95615


In [70]:
# define the feature extraction method
from pandas import DataFrame
from typing import Iterable

def extract_hrv_features_from_rri(
    rr: Iterable[float],
    fs = 4,
    ws = 300,
  ) -> dict[str, float]:
  '''
  Extracts HRV features from a series of RR intervals.
  RR intervals are in milliseconds.

  ### Parameters:
  - rr (Iterable[float]): A series of RR intervals in milliseconds.
  - fs (int): The sampling frequency in Hz.
  - ws (int): The window size in seconds.
  '''
  import numpy as np
  from scipy.interpolate import interp1d
  from scipy.stats import skew, kurtosis
  from scipy.signal import welch
  from antropy import sample_entropy, higuchi_fd

  rr = np.array(rr)
  mrr = np.mean(rr)
  sdrr = np.std(rr)
  sd = np.diff(rr)
  sdsd = np.std(sd)
  rmssd = np.mean(sd ** 2) ** 0.5
  asd = abs(sd)
  rel_rr = sd / (rr[1:] + rr[:-1]) * 2
  rel_sd = np.diff(rel_rr)
  rel_sdrr = np.std(rel_rr)
  rel_rmssd = np.mean(rel_sd ** 2) ** 0.5
  int_rr_x = np.cumsum(rr) / 1000
  int_rr_x_new = np.arange(1, max(int_rr_x), 1 / fs)
  int_rr = interp1d(
    x=int_rr_x, y=rr, copy=False,
    kind='cubic', fill_value='extrapolate',
  )(int_rr_x_new)
  fr, ps = welch(
    x=int_rr, fs=fs,
    nperseg=min(ws * fs, len(int_rr_x_new)),
  )
  cond_vlf = (fr >= 0.003) & (fr <= 0.04)
  cond_lf = (fr >= 0.04) & (fr <= 0.15)
  cond_hf = (fr >= 0.15) & (fr <= 0.4)
  vlf = np.trapz(y=ps[cond_vlf], x=fr[cond_vlf])
  lf = np.trapz(y=ps[cond_lf], x=fr[cond_lf])
  hf = np.trapz(y=ps[cond_hf], x=fr[cond_hf])
  nu = lf + hf
  tp = vlf + nu
  return {
    'MEAN_RR': mrr,
    'MEDIAN_RR': np.median(rr),
    'SDRR': sdrr,
    'RMSSD': rmssd,
    'SDSD': sdsd,
    'SDRR_RMSSD': sdrr / rmssd,
    'HR': 60000 / mrr,
    'pNN25': np.mean(asd > 25) * 100,
    'pNN50': np.mean(asd > 50) * 100,
    'KURT': kurtosis(rr),
    'SKEW': skew(rr),
    'SD1': (2 ** -0.5) * sdsd,
    'SD2': (2 * (sdrr ** 2) - 0.5 * (sdsd ** 2)) ** 0.5,
    'MEAN_REL_RR': np.mean(rel_rr),
    'MEDIAN_REL_RR': np.median(rel_rr),
    'SDRR_REL_RR': rel_sdrr,
    'RMSSD_REL_RR': rel_rmssd,
    'SDSD_REL_RR': np.std(rel_sd),
    'SDRR_RMSSD_REL_RR': rel_sdrr / rel_rmssd,
    'KURT_REL_RR': kurtosis(rel_rr),
    'SKEW_REL_RR': skew(rel_rr),
    'VLF': vlf,
    'VLF_PCT': vlf / tp * 100,
    'LF': lf,
    'LF_PCT': lf / tp * 100,
    'LF_NU': lf / nu * 100,
    'HF': hf,
    'HF_PCT': hf / tp * 100,
    'HF_NU': hf / nu * 100,
    'TP': tp,
    'LF_HF': lf / hf,
    'HF_LF': hf / lf,
    'sampen': sample_entropy(x=rr, order=0),
    'higuci': higuchi_fd(rr),
  }

def _extract_hrv_features_from_subject(
  data: DataFrame,
  fs = 4,
  ws = 300,
):
  import numpy as np

  s = fs * ws # sample size
  for win_end in np.arange(s, len(data) + 1):
    window = data[win_end-s: win_end]
    features = extract_hrv_features_from_rri(window['rri'], fs=fs, ws=ws)
    features['datasetId'] = 2
    features['condition'] = window['label'].value_counts().idxmax()
    yield features

def extract_hrv_features_from_subject(
  data: DataFrame,
  fs = 4,
  ws = 300,
):
  '''
  Extracts HRV features from a subject data. (`DataFrame {label, rri}`)

  RR intervals are in milliseconds.

  ### Parameters:
  - data (DataFrame): A dict-like object with series of `label` and `rri` data.
  - fs (int): The sampling frequency in Hz.
  - ws (int): The window size in seconds.
  '''
  return DataFrame(_extract_hrv_features_from_subject(
    data=data,
    fs=fs,
    ws=ws,
  ))

In [ ]:
# transform label and RRI data
from multiprocess.pool import Pool
from tqdm import tqdm

SAMPLE_FREQ_HZ = 4
WINDOW_SIZE_SEC = 300

with Pool() as pool:
  subjects = list(map(lambda t: t[1], MERGED_DATA.groupby('subject')))
  TRANSFORMED_DATA = (
    pd.concat(
      objs=tqdm(
        pool.imap(
          func=extract_hrv_features_from_subject,
          iterable=subjects,
        ),
        total=len(subjects),
      ),
      copy=False,
      ignore_index=True,
    )
    .sample(n=41000, random_state=123)
    .reset_index(drop=True)
  )
  TRANSFORMED_DATA['condition'] = TRANSFORMED_DATA['condition'].astype('category')
  subjects = None
  display(TRANSFORMED_DATA)

In [83]:
# save the compressed and transformed data
from zipfile import ZipFile, ZIP_DEFLATED

with ZipFile(
  file=f'{TRANSFORMED_DATA_OUTPUT_NAME}.zip',
  mode='w',
  compression=ZIP_DEFLATED,
  compresslevel=9,
) as comp_file:
  TRANSFORMED_DATA.to_csv(
    index=False,
    path_or_buf=comp_file.open(
      name=f'{TRANSFORMED_DATA_OUTPUT_NAME}.csv',
      mode='w',
    ),
  )
  print(f'Transformed data has been saved to {TRANSFORMED_DATA_OUTPUT_NAME}.zip')

Transformed data has been saved to ../datasets/swell/data/final/test-custom-1.zip


In [79]:
def mem_mb(data):
  return data.memory_usage(deep=True).sum() / 1024 / 1024

[mem_mb(d) for d in [LABEL_DATA, RRI_DATA, MERGED_DATA, TRANSFORMED_DATA]]

[0.08281803131103516,
 28.89668846130371,
 30.712042808532715,
 10.987702369689941]